In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from pathlib import Path
pd.set_option('display.max_columns', 500)

import allensdk.brain_observatory.behavior.behavior_project_cache as bpc

import allensdk
import pkg_resources
print('allensdk version 2.10.2 or higher is required, you have {} installed'.format(pkg_resources.get_distribution("allensdk").version))

%matplotlib notebook
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
 # bpc.VisualBehaviorOphysProjectCache.

In [5]:
# choose a location on your file system to cache NWB files as they are loaded:
my_cache_dir = Path("G:/allensdk")

bc = bpc.VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=my_cache_dir)
# bpc.VisualBehaviorOphysProjectCache.construct_local_manifest(bc)

behavior_session_table = bc.get_behavior_session_table()

In [6]:
len(behavior_session_table.mouse_id.unique())

In [7]:
mouse_id_list = behavior_session_table.mouse_id.unique()
mouse_id_list.sort()
print(mouse_id_list)

In [13]:
# mouse_id_list = [445002,403491,412036,412366,421136,423606,425493,425496,429952,431023]
# mouse_id_list = [445002,403491,412036,412366,421136]
mouse_id_list = behavior_session_table.mouse_id.unique()
mouse_id_list.sort()
all_mouse_table = behavior_session_table.copy()
# this_mouse_table = behavior_session_table.query('mouse_id == @mouse_id and project_code == "VisualBehavior"').sort_values(by = 'session_type') 
# all_mouse_table = behavior_session_table[behavior_session_table['mouse_id'].isin(mouse_id_list)]
all_mouse_table = all_mouse_table.query('project_code == "VisualBehavior" and (session_type == "OPHYS_6_images_B" or session_type == "OPHYS_4_images_B" or session_type == "OPHYS_3_images_A" or session_type == "OPHYS_1_images_A" )').sort_values(['mouse_id', 'session_type'], ascending=[True, True])
# only active sessions

# all_mouse_table = all_mouse_table.query('project_code == "VisualBehavior" and behavior_type == "active_behavior"').sort_values(['mouse_id', 'session_type'], ascending=[True, True])

# all_mouse_table = all_mouse_table.query('project_code == "VisualBehavior" and (session_type == "OPHYS_6_images_B" or session_type == "OPHYS_4_images_B" or session_type == "OPHYS_3_images_A" or session_type == "OPHYS_1_images_A" )').sort_values(['mouse_id', 'session_type'], ascending=[True, True])
all_mouse_table

# for mouse_id in all_mouse_table['mouse_id'].unique():
#     this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
#     if len(this_mouse_table.index) != 4 :
#         all_mouse_table = all_mouse_table.drop(this_mouse_table.index.values)

In [ ]:
behavior_session_ids = all_mouse_table.index.values
behavior_session_dict = {}
for behavior_session_id in behavior_session_ids:
    print(behavior_session_id)
    behavior_session_dict[behavior_session_id] = bc.get_behavior_session(behavior_session_id)

In [13]:
# table = all_mouse_table.copy()
# for mouse_id in table['mouse_id'].unique():
#     this_mouse_table = table.query('mouse_id == @mouse_id')
#     if len(this_mouse_table.index) != 4 :
#         print(this_mouse_table.index.values)
#         table = table.drop(this_mouse_table.index.values)
#     elif len(this_mouse_table.index) > 4:
#         print('kok')
#         table = pd.concat(table.loc[table.mouse_id != mouse_id], table.loc[table.mouse_id == mouse_id].drop_duplicates(subset='session_type', keep="last")).sort_values(['mouse_id'])
# #         table[table['mouse_id'] == mouse_id].drop_duplicates(subset='session_type', keep="last")
# table
        

In [14]:
behavior_performance_table = pd.DataFrame(
    [behavior_session_dict[behavior_session_id].get_performance_metrics() for behavior_session_id in behavior_session_ids]
).set_index(behavior_session_ids)
all_mouse_table = all_mouse_table.merge(
    behavior_performance_table,
    left_index = True,
    right_index = True,
)
# all_mouse_table

In [15]:
all_mouse_table

In [16]:
all_mouse_table.cre_line.unique()

In [17]:
# behavior_session_id = 775378326

for behavior_session_id in all_mouse_table.index:
    behavior_session = behavior_session_dict[behavior_session_id]
    stimulus_presentations = behavior_session.stimulus_presentations.copy()
# calculate average reaction time
    changed_stimulus_timestamps = stimulus_presentations['start_time'][stimulus_presentations['is_change'] == True].to_numpy()
    reaction_time_list = np.array([])
    for time_stamp in changed_stimulus_timestamps:
        idx = (behavior_session.licks['timestamps'] >= time_stamp).idxmax()
        lick_after_changed_timestamps = behavior_session.licks['timestamps'][idx]
        reaction_time = lick_after_changed_timestamps - time_stamp
        if reaction_time < 0.75000 and reaction_time > 0.15000:
            reaction_time_list = np.append(reaction_time_list,reaction_time)
    average_reaction_time = reaction_time_list.mean()
    all_mouse_table.loc[behavior_session_id,'average_reaction_time'] = average_reaction_time
    


#     calculate running speed
    all_mouse_table.loc[behavior_session_id, 'mean_running_speed'] = behavior_session.running_speed['speed'].mean()

# all_mouse_table

In [18]:
beahvior_session_table.stimulus_mean_hit_rate_engaged?

## individual mice plots

In [19]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['prior_exposures_to_image_set'].to_numpy()
    y = this_mouse_table['average_reaction_time'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')

#     newax = ax.twiny()
#     newax.set_xlim(ax.get_xlim())
#     ax.set_xticks(this_mouse_table['prior_exposures_to_image_set'])
#     fig.subplots_adjust(bottom=0.5)
#     ax.set_xticklabels(this_mouse_table['prior_exposures_to_image_set'].astype(int))
#     newax.xaxis.set_label_position('bottom')
#     newax.xaxis.set_ticks_position('bottom')
#     newax.set_xticks(ax.get_xticks())
#     newax.set_xticklabels(this_mouse_table['session_type'].values,rotation = 30, ha='right',y=-0.02)
    # ax.tick_params( axis='x', which='minor')
    ax.set(xlabel='prior_exposores', ylabel='average_reaction_time')

ax.set_title('exposure/ average reaction time plot')
fig.savefig('exposure_average_reaction_time_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [20]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['prior_exposures_to_image_set'].to_numpy()
    y = this_mouse_table['mean_hit_rate'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')
    ax.set(xlabel='prior_exposores', ylabel='average_hit_rate')

ax.set_title('exposure/average_hit_rate plot')
fig.savefig('exposure_hit_rate_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [21]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['prior_exposures_to_image_set'].to_numpy()
    y = this_mouse_table['mean_false_alarm_rate'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')
    ax.set(xlabel='prior_exposores', ylabel='false_alarm_rate')

ax.set_title('exposure/false_alarm_rate plot')
fig.savefig('exposure_mean_false_alarm_rate_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [22]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['mean_running_speed'].to_numpy()
    y = this_mouse_table['average_reaction_time'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')
    ax.set(xlabel='mean_running_speed', ylabel='average_reaction_time')

ax.set_title('mean_running_speed/average_reaction_time plot')
fig.savefig('mean_running_speed_average_reaction_time_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [23]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['mean_running_speed'].to_numpy()
    y = this_mouse_table['mean_hit_rate'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')
    ax.set(xlabel='mean_running_speed', ylabel='average_hit_rate')

ax.set_title('mean_running_speed/average_hit_rate plot')
fig.savefig('mean_running_speed_average_hit_rate_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [24]:
fig, ax = plt.subplots(figsize = (12,10))

for mouse_id in all_mouse_table['mouse_id'].unique():
    this_mouse_table = all_mouse_table.query('mouse_id == @mouse_id')
    x = this_mouse_table['mean_running_speed'].to_numpy()
    y = this_mouse_table['mean_false_alarm_rate'].to_numpy()
    lists = sorted(zip(*[x, y]))
    new_x, new_y = list(zip(*lists))
    ax.plot(new_x, new_y,'-o')
    ax.set(xlabel='mean_running_speed', ylabel='average_false_alarm_rate')

ax.set_title('mean_running_speed/average_false_alarm_rate plot')
fig.savefig('mean_running_speed_average_false_alarm_rate_plot.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [25]:
session_types = ['OPHYS_1_images_A', 'OPHYS_3_images_A', 'OPHYS_4_images_B', 'OPHYS_6_images_B']
avg_reaction_time_all = np.array([])
avg_running_speed = np.array([])
avg_hit_rate = np.array([])
avg_false_alarm_rate = np.array([])
avg_prior_exposure = np.array([])
for session in session_types:
    avg_reaction_time_all = np.append(avg_reaction_time_all, all_mouse_table.average_reaction_time[all_mouse_table['session_type'] == session].mean())
    avg_running_speed = np.append(avg_running_speed, all_mouse_table.mean_running_speed[all_mouse_table['session_type'] == session].mean())
    avg_hit_rate = np.append(avg_hit_rate, all_mouse_table.mean_hit_rate[all_mouse_table['session_type'] == session].mean())
    avg_false_alarm_rate = np.append(avg_false_alarm_rate, all_mouse_table.mean_false_alarm_rate[all_mouse_table['session_type'] == session].mean())
    avg_prior_exposure = np.append(avg_prior_exposure, all_mouse_table.prior_exposures_to_image_set[all_mouse_table['session_type'] == session].mean())

print(avg_reaction_time_all)
print(avg_running_speed)
print(avg_hit_rate)
print(avg_false_alarm_rate)
print(avg_prior_exposure)

In [26]:
# my_sessions = session_types
# my_x = avg_prior_exposure
# print(my_sessions)
# print(my_x)
# lists = sorted(zip(*[my_x, my_sessions]))
# my_new_x, my_new_sessions = list(zip(*lists))
# print(my_new_sessions)
# print(my_new_x)

In [27]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_prior_exposure
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_prior_exposure
y = avg_reaction_time_all
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')

ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.02)
newax.set_xlim(ax.get_xlim())

ax.set(xlabel='avg_prior_exposure', ylabel='avg_reaction_time')
# plt.xticks(np.arange(0, avg_prior_exposure.max()+1, 1.0))
ax.set_title('avg_prior_exposure/avg_reaction_time plot')
fig.savefig('avg_prior_exposure_avg_reaction_time.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [28]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_prior_exposure
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_prior_exposure
y = avg_hit_rate
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')
ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.02)
newax.set_xlim(ax.get_xlim())

ax.set(xlabel='avg_prior_exposure', ylabel='avg_hit_rate')
# plt.xticks(np.arange(0, avg_prior_exposure.max()+1, 1.0))
ax.set_title('avg_prior_exposure/avg_hit_rate plot')
fig.savefig('avg_prior_exposure_avg_hit_rate.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [29]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_prior_exposure
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_prior_exposure
y = avg_false_alarm_rate
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')

ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.02)
newax.set_xlim(ax.get_xlim())


ax.set(xlabel='avg_prior_exposure', ylabel='avg_false_alarm_rate')
# plt.xticks(np.arange(0, avg_prior_exposure.max()+1, 1.0))
ax.set_title('avg_prior_exposure/avg_false_alarm_rate plot')
fig.savefig('avg_prior_exposure_avg_false_alarm_rate.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [30]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_running_speed
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_running_speed
y = avg_reaction_time_all
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')
ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.06)
newax.set_xlim(ax.get_xlim())



ax.set(xlabel='avg_running_speed', ylabel='avg_reaction_time')
# plt.xticks(np.arange(avg_running_speed.min(), avg_running_speed.max(), 0.5))
ax.set_title('avg_running_speed/avg_reaction_time plot')
fig.savefig('avg_running_speed_avg_reaction_time.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [31]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_running_speed
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_running_speed
y = avg_hit_rate
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')
ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.06)
newax.set_xlim(ax.get_xlim())

ax.set(xlabel='avg_running_speed', ylabel='avg_hit_rate')
# plt.xticks(np.arange(avg_running_speed.min(), avg_running_speed.max(), 0.5))
ax.set_title('avg_running_speed/avg_hit_rate plot')
fig.savefig('avg_running_speed_avg_hit_rate.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [32]:
fig, ax = plt.subplots(figsize = (12,10))

x = avg_running_speed
y = avg_false_alarm_rate
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')
ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.06)
newax.set_xlim(ax.get_xlim())

ax.set(xlabel='avg_running_speed', ylabel='avg_false_alarm_rate')
# plt.xticks(np.arange(avg_running_speed.min(), avg_running_speed.max(), 0.5))
ax.set_title('avg_running_speed/avg_false_alarm_rate plot')
fig.savefig('avg_running_speed_avg_false_alarm_rate.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [33]:
len(all_mouse_table['mouse_id'].unique())

## Adding pupil area

In [45]:
for behavior_session_id in all_mouse_table.index:
    behavior_session = behavior_session_dict[behavior_session_id]
    
    ophys_exp_id = all_mouse_table.loc[behavior_session_id,'ophys_experiment_id']
    # print(ophys_exp_id)
    behavior_exp = bc.get_behavior_ophys_experiment(ophys_exp_id)
    if behavior_exp.eye_tracking is not None:
        avg_pupil_area = behavior_exp.eye_tracking.pupil_area.mean()    
        all_mouse_table.loc[behavior_session_id,'avg_pupil_area'] = avg_pupil_area

all_mouse_table

In [46]:
session_types = ['OPHYS_1_images_A', 'OPHYS_3_images_A', 'OPHYS_4_images_B', 'OPHYS_6_images_B']
avg_pupil_area_all = np.array([])
for session in session_types:
    avg_pupil_area_all = np.append(avg_pupil_area_all, all_mouse_table.avg_pupil_area[all_mouse_table['session_type'] == session].mean())

print(avg_pupil_area_all)

In [47]:
fig, ax = plt.subplots(figsize = (12,10))

x = avg_running_speed
y = avg_pupil_area_all
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')
ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.06)
newax.set_xlim(ax.get_xlim())

ax.set(xlabel='avg_running_speed', ylabel='avg_pupil_area')
# plt.xticks(np.arange(avg_running_speed.min(), avg_running_speed.max(), 0.5))
ax.set_title('avg_running_speed/avg_pupil_area plot')
fig.savefig('avg_running_speed_avg_pupil_area_all.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [48]:
fig, ax = plt.subplots(figsize = (12,10))

my_sessions = session_types
my_x = avg_prior_exposure
lists = sorted(zip(*[my_x, my_sessions]))
my_new_x, sorted_sessions = list(zip(*lists))

x = avg_prior_exposure
y = avg_pupil_area_all
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')

ax.set_yticks(np.linspace(y.min(), y.max(), 10))
ax.set_xticks(new_x)
ax.set_xticklabels(np.around(new_x,2))
newax = ax.twiny()

fig.subplots_adjust(bottom=0.5)
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(sorted_sessions,rotation = 30, ha='right',y=-0.02)
newax.set_xlim(ax.get_xlim())


ax.set(xlabel='avg_prior_exposure', ylabel='avg_pupil_area')
# plt.xticks(np.arange(0, avg_prior_exposure.max()+1, 1.0))
ax.set_title('avg_prior_exposure/avg_pupil_area plot')
fig.savefig('avg_prior_exposure_avg_pupil_area_all.png', dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [42]:
behavior_session_id = 775378326

ophys_exp_id = all_mouse_table.loc[behavior_session_id,'ophys_experiment_id']
# print(ophys_exp_id)
behavior_exp = bc.get_behavior_ophys_experiment(ophys_exp_id)

behavior_exp.eye_tracking.pupil_area.mean()

# stimulus_presentations = behavior_session.stimulus_presentations.copy()
# # calculate average reaction time
# changed_stimulus_timestamps = stimulus_presentations['start_time'][stimulus_presentations['is_change'] == True].to_numpy()
# reaction_time_list = np.array([])
# lick_time_list = np.array([])
# for time_stamp in changed_stimulus_timestamps:
#     idx = (behavior_session.licks['timestamps'] >= time_stamp).idxmax()
#     lick_after_changed_timestamps = behavior_session.licks['timestamps'][idx]
#     reaction_time = lick_after_changed_timestamps - time_stamp
# #     if(lick_after_changed_timestamps in behavior_session.rewards['timestamps'].unique()):
#     if reaction_time < 0.75000 and reaction_time > 0.15000:
#         reaction_time_list = np.append(reaction_time_list,reaction_time)
#         lick_time_list = np.append(lick_time_list,lick_after_changed_timestamps)
        

# np.set_printoptions(suppress=True)
# print(reaction_time_list)



In [ ]:
all_mouse_table['mean_running_speed'].to_numpy()

In [ ]:
itemindex = np.where(reaction_time_list==0)
itemindex

In [ ]:
print(lick_time_list[itemindex])

In [ ]:
print(reaction_time_list[reaction_time_list<0.250])

In [ ]:
np.absolute(behavior_session.running_speed['speed']).mean()

In [ ]:
time_stamp = 2181.47082
idx = (behavior_session.licks['timestamps'] >= time_stamp).idxmax()
lick_after_changed_timestamps = behavior_session.licks['timestamps'][idx]
reaction_time = lick_after_changed_timestamps - time_stamp
print(lick_after_changed_timestamps)
print(reaction_time)

In [ ]:
print(lick_time_list)

In [ ]:
changed_stimulus_timestamps

In [ ]:
behavior_session.licks['timestamps'][behavior_session.licks['timestamps'] >= 993.31147]

In [ ]:
pd.set_option('display.max_rows', 109)
behavior_session.rewards.tail(10)

In [ ]:
# print(lick_time_list - behavior_session.rewards['timestamps'])
A = lick_time_list
B = behavior_session.rewards['timestamps']
A[B[np.searchsorted(B,A)] !=  A]

In [ ]:
len(behavior_session.rewards)

In [ ]:
331.25611 in behavior_session.rewards['timestamps'].unique()
# type(behavior_session.rewards['timestamps'])

In [ ]:
behavior_session.running_speed

In [ ]:
fig, ax = plt.subplots(figsize = (12,10))

x = this_mouse_table_ophys['prior_exposures_to_image_set'].to_numpy()
y = this_mouse_table_ophys['first_lick_after_stimulus'].to_numpy()
lists = sorted(zip(*[x, y]))
new_x, new_y = list(zip(*lists))
ax.plot(new_x, new_y,'-o')

newax = ax.twiny()
newax.set_xlim(ax.get_xlim())

ax.set_xticks(this_mouse_table_ophys['prior_exposures_to_image_set'])
fig.subplots_adjust(bottom=0.5)

ax.set_xticklabels(this_mouse_table_ophys['prior_exposures_to_image_set'].astype(int))
newax.xaxis.set_label_position('bottom')
newax.xaxis.set_ticks_position('bottom')
newax.set_xticks(ax.get_xticks())
newax.set_xticklabels(this_mouse_table_ophys['session_type'].values,rotation = 30, ha='right',y=-0.02)
# ax.tick_params( axis='x', which='minor')
ax.set(xlabel='prior_exposores', ylabel='first_lick_after_stimulus')
ax.set_title('exposure/first_lick_after_stimulus plot for mouse_id= {}'.format(mouse_id))
fig.savefig('exposure_first_lick_after_stimulus_plot for mouse_id= {}.png'.format(mouse_id), dpi=300,bbox_inches = "tight")
fig.tight_layout()

In [ ]:
pd.set_option('display.max_rows', 40)
stimulus_presentations